# Anova for our analysis
We run an anova to compare ethnicities and emotions, each combination of those. We have to leave out the asians unfortunately:(

# Load in packages 

In [29]:
import numpy as np
import mne
from pathlib import Path
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.formula.api import mixedlm

## Load in the data and epochs

In [30]:
# ALL FROM LAURAS GROUP LEVEL ANALYSIS
# Define the directory
data_path = Path("/Users/lina/Documents/GitHub/ore_EEG/epochs")

# Initialize an empty list to store epochs objects
all_epochs = []

# Iterate over your saved files and load them into epochs objects
for participant in ["own_sub1", "own2_ah"]:
    # Load epochs data from each file
    epochs = mne.read_epochs(data_path / f"epochs_{participant}-epo.fif", verbose=False, preload=True)
    
    # Only keep EEG channels
    epochs.pick_types(eeg=True)
    
    # Append the loaded epochs object to the list
    all_epochs.append(epochs)

# Check the type and length of the list of epochs objects
print(type(all_epochs))  # We have now created a list of epochs objects
print(len(all_epochs))    # We have 2 epochs objects in the list

# Access the first epochs object in the list
print(type(all_epochs[0]))  # We can access the first epochs object in the list which is an Epochs object

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
<class 'list'>
2
<class 'mne.epochs.EpochsFIF'>


## Define channels and time used for the anova

In [2]:
# chosen channels for anova
picks = ["Oz", "P7"] # just random 

# time window for anova
tmin = 0.1
tmax = 0.2

## Define Factors

In [3]:
# make lists for labels in conditions
ethnicity_labels = ['white', 'black']
emotion_labels = ['happy', 'neutral', 'sad']
data = []

# WE want combinations of ethnicity and emotion, so we loop over all of the epochs conditions
## ADDED SUBJECT HERE
for i, epochs in enumerate(all_epochs):
    for ethnicity in ethnicity_labels:
        for emotion in emotion_labels:
                current_epochs = epochs[f'{emotion}/{ethnicity}']
                data_current = current_epochs.get_data(picks=picks, tmin=tmin, tmax=tmax)
                data_mean = np.mean(data_current, axis=(0, 1, 2))
                # append the mean data, labels, and subject ID to the list
                data.append([i, ethnicity, emotion, data_mean])

# Make to dataframe
df = pd.DataFrame(data, columns=['Subject', 'Ethnicity', 'Emotion', 'Mean'])

# see hot it looks
print(df)


    Subject Ethnicity  Emotion      Mean
0         0     white    happy  0.000002
1         0     white  neutral  0.000002
2         0     white      sad  0.000002
3         0     black    happy  0.000002
4         0     black  neutral  0.000002
5         0     black      sad  0.000002
6         1     white    happy  0.000001
7         1     white  neutral  0.000001
8         1     white      sad  0.000002
9         1     black    happy  0.000002
10        1     black  neutral  0.000001
11        1     black      sad  0.000002


# Perform anova

In [8]:
# Fit model with subject as group level
model = mixedlm('Mean ~ C(Ethnicity) * C(Emotion)', data=df, groups=df['Subject'])
anova_result = model.fit()

# Print the summary of the model
print(anova_result.summary())


                        Mixed Linear Model Regression Results
Model:                        MixedLM           Dependent Variable:           Mean    
No. Observations:             24                Method:                       REML    
No. Groups:                   2                 Scale:                        0.0000  
Min. group size:              12                Log-Likelihood:               252.8268
Max. group size:              12                Converged:                    Yes     
Mean group size:              12.0                                                    
--------------------------------------------------------------------------------------
                                            Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------------------------------------
Intercept                                    0.000    0.000  7.459 0.000  0.000  0.000
C(Ethnicity)[T.white]                       -0.000    0.000 -4.021 0

/Applications/anaconda3/lib/python3.11/site-packages/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
